In [32]:


import os
import pandas as pd

# Optional helper (reuse across notebooks)
def get_path(*parts):
    return os.path.join('..', *parts)


# --- 1. Load both CSV files and set "Country" as index ---
df_hours1 = pd.read_csv(get_path('Data', 'Working_hours_Penn.csv'))
df_hours2 = pd.read_csv(get_path('Data', 'Working_hours_annualy_average.csv'))

df_hours1.set_index("Country", inplace=True)
df_hours2.set_index("Country", inplace=True)


# 2. Merge the dataframes on Country using an outer join
#    We'll give the second dataframe's columns a suffix "_penn"
df_merged = df_hours1.join(df_hours2, how="outer", lsuffix="", rsuffix="_penn")

# 3. Combine each pair of year columns into a single column
#    (fill missing from df_hours1 with df_hours2, then drop the second column)
years = range(1990, 2026)  # adjust if needed
for y in years:
    col_a = str(y)             # e.g. "1990"
    col_b = f"{y}_penn"        # e.g. "1990_penn"
    
    if col_a in df_merged.columns and col_b in df_merged.columns:
        # Fill missing from col_a with col_b
        df_merged[col_a] = df_merged[col_a].fillna(df_merged[col_b])
        df_merged.drop(columns=[col_b], inplace=True)
    elif col_b in df_merged.columns:
        # If col_a doesn't exist, rename col_b -> col_a
        df_merged.rename(columns={col_b: col_a}, inplace=True)

# 4. Sort alphabetically by "Country" (the index), then reset the index
df_merged.sort_index(inplace=True)
df_merged.reset_index(inplace=True)

# 5. Clean up "~" and round numeric columns
columns_to_clean = [col for col in df_merged.columns if col != "Country"]
for col in columns_to_clean:
    # Convert to string to remove "~" and handle 'nan'
    df_merged[col] = (df_merged[col].astype(str)
                                 .str.replace('~', '')  # remove tilde
                                 .replace('nan', '')    # handle 'nan' strings
                     )
    # Convert to numeric (float), coercing errors to NaN
    df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')
    # Round to nearest whole number
    df_merged[col] = df_merged[col].round(0)
    # Convert to an integer type (nullable Int64 if you want to keep NaN)
    df_merged[col] = df_merged[col].astype('Int64')

# 6. Save the merged, cleaned file to a single CSV
df_merged.to_csv("merged_single_column_hours_cleaned.csv", index=False)
output_path = get_path('Result', 'Merged_single_column_hours_cleaned_TEST.csv')
df_merged.to_csv(output_path, index=False)

print(f"✅ Merged file saved to: {output_path}")


✅ Merged file saved to: ../Result/Merged_single_column_hours_cleaned_TEST.csv


In [34]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Paths to the two datasets
penn_path   = get_path('Data', 'Working_hours_Penn.csv')
merged_path = get_path('Result', 'Merged_single_column_hours_cleaned_TEST.csv')


# 1. Load original Penn hours and merged cleaned hours
df_penn   = pd.read_csv(penn_path)
df_merged = pd.read_csv(merged_path)

# 2. Normalize country column name and year columns to numeric
df_penn.columns = df_penn.columns.str.strip().str.lower()
df_merged.columns = df_merged.columns.str.strip().str.lower()

# Melt Penn (if still wide) to long form
if 'year' not in df_penn.columns:
    df_penn = df_penn.melt(id_vars=['country'], var_name='year', value_name='hours_penn')
df_penn['year'] = pd.to_numeric(df_penn['year'], errors='coerce')

# Melt merged (if still wide) to long form
if 'year' not in df_merged.columns:
    df_merged = df_merged.melt(id_vars=['country'], var_name='year', value_name='hours_merged')
df_merged['year'] = pd.to_numeric(df_merged['year'], errors='coerce')

# 3. Identify countries in common
common_countries = sorted(set(df_penn['country']).intersection(df_merged['country']))
print(f"Found {len(common_countries)} countries in common.")

# A: as a plain text file, one country per line
# Save common countries list
txt_path = get_path('Result', 'common_countries.txt')
with open(txt_path, 'w') as f:
    for c in common_countries:
        f.write(c + '\n')

print(f"✅ Saved common countries to: {txt_path}")

# 4. Create a multi-page PDF to save one plot per country
output_pdf = get_path('Result', 'Plots', 'Hours_Comparison_by_Country.pdf')
with PdfPages(output_pdf) as pdf:
    for country in common_countries:
        # filter data
        d1 = df_penn[df_penn['country'] == country].sort_values('year')
        d2 = df_merged[df_merged['country'] == country].sort_values('year')

        # skip if no data
        if d1.empty or d2.empty:
            continue

        # Plot
        plt.figure(figsize=(8, 4))
        plt.plot(d1['year'], d1['hours_penn'],   marker='o', label='Penn Hours')
        plt.plot(d2['year'], d2['hours_merged'], marker='x', label='Merged Hours')
        plt.title(f'Working Hours Comparison: {country}')
        plt.xlabel('Year')
        plt.ylabel('Annual Hours Worked')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        pdf.savefig()
        plt.close()

print(f"Saved comparison plots for {len(common_countries)} countries to {output_pdf}")


Found 69 countries in common.
✅ Saved common countries to: ../Result/common_countries.txt
Saved comparison plots for 69 countries to ../Result/Plots/Hours_Comparison_by_Country.pdf


In [36]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# List of countries to apply trend extrapolation
countries_to_modify = [
    "Argentina", "Australia", "Austria", "Bangladesh", "Belgium", "Brazil",
    "Bulgaria", "Cambodia", "Canada", "Chile", "China", "China, Hong Kong SAR",
    "Colombia", "Costa Rica", "Croatia", "Cyprus", "Czech Republic", "Denmark",
    "Dominican Republic", "Ecuador", "Estonia", "Finland", "France", "Germany",
    "Greece", "Hungary", "Iceland", "India", "Indonesia", "Ireland", "Israel",
    "Italy", "Jamaica", "Japan", "Latvia", "Lithuania", "Luxembourg", "Malaysia",
    "Malta", "Mexico", "Myanmar", "Netherlands", "New Zealand", "Norway",
    "Pakistan", "Peru", "Philippines", "Poland", "Portugal", "Republic of Korea",
    "Romania", "Russian Federation", "Singapore", "Slovakia", "Slovenia",
    "South Africa", "Spain", "Sri Lanka", "Sweden", "Switzerland", "Taiwan",
    "Thailand", "Trinidad and Tobago", "Turkey", "United Kingdom", "United States",
    "Uruguay", "Venezuela (Bolivarian Republic of)", "Viet Nam"
]

# 1) Load the merged
# ✅ Define the correct path to your merged file
path = get_path('Result', 'Merged_single_column_hours_cleaned_TEST.csv')
df_wide = pd.read_csv(path)

# 2) Melt to long form: Country, year, hours
df_long = df_wide.melt(id_vars='Country', var_name='year', value_name='hours')
df_long['year'] = pd.to_numeric(df_long['year'], errors='coerce')

# 3) For each country in the list, fit a regression on years ≤2019 and predict 2020–2023
global_cutoff = 2019
def fill_with_trend(g):
    train = g[(g['year'] <= global_cutoff) & g['hours'].notna()]
    if len(train) < 2:
        return g
    X = train[['year']].values
    y = train['hours'].values
    model = LinearRegression().fit(X, y)
    future_years = np.arange(global_cutoff + 1, 2024)
    Xf = future_years.reshape(-1, 1)
    yf = model.predict(Xf)
    df_pred = pd.DataFrame({
        'Country': g.name,
        'year': future_years,
        'hours': yf
    })
    hist = g[g['year'] <= global_cutoff]
    return pd.concat([hist, df_pred], ignore_index=True)

def process_country(g):
    if g.name in countries_to_modify:
        return fill_with_trend(g)
    else:
        return g

# Apply only to selected countries
df_filled = df_long.groupby('Country', group_keys=False).apply(process_country)

# 4) Pivot back to wide form
df_corrected = df_filled.pivot(index='Country', columns='year', values='hours').reset_index()

# 5) Round to nearest integer
years = [c for c in df_corrected.columns if isinstance(c, (int, float))]
df_corrected[years] = df_corrected[years].round(0).astype('Int64')

# 6)Save corrected CSV
out_path = get_path('Result', 'Hours_extrapolated_1990_2023_selected.csv')
df_corrected.to_csv(out_path, index=False)

print(f"✅ Extrapolated hours for selected countries saved to: {out_path}")


✅ Extrapolated hours for selected countries saved to: ../Result/Hours_extrapolated_1990_2023_selected.csv


/var/folders/pj/s_kq924572g7znglnlstjmn40000gn/T/ipykernel_52871/2271354249.py:58: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled = df_long.groupby('Country', group_keys=False).apply(process_country)


In [38]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# paths
old_path = get_path('Result', 'Merged_single_column_hours_cleaned_TEST.csv')
new_path = get_path('Result', 'Hours_extrapolated_1990_2023_selected.csv')

# 1) load and melt old vs new
df_old = pd.read_csv(old_path).melt(
    id_vars='Country', var_name='year', value_name='old_hours'
)
df_old['year'] = pd.to_numeric(df_old['year'], errors='coerce')

df_new = pd.read_csv(new_path).melt(
    id_vars='Country', var_name='year', value_name='new_hours'
)
df_new['year'] = pd.to_numeric(df_new['year'], errors='coerce')

# 2) restrict to common Country×year pairs
df_cmp = (
    df_old.merge(df_new, on=['Country','year'], how='inner')
    .sort_values(['Country','year'])
)

# 3) plot each country into a multipage PDF
out_pdf = get_path('Result','Plots', 'Hours_Comparison_Old_vs_Extrapolated.pdf')
with PdfPages(out_pdf) as pdf:
    for country, grp in df_cmp.groupby('Country'):
        plt.figure(figsize=(8,4))
        plt.plot(grp['year'], grp['old_hours'], marker='o', label='Old (raw/merged)')
        plt.plot(grp['year'], grp['new_hours'], marker='x', label='New (extrapolated)')
        plt.title(f'Hours Worked Comparison: {country}')
        plt.xlabel('Year')
        plt.ylabel('Annual Hours Worked')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        pdf.savefig()
        plt.close()

print(f"Saved updated comparison plots to:\n  {out_pdf}")


Saved updated comparison plots to:
  ../Result/Plots/Hours_Comparison_Old_vs_Extrapolated.pdf
